In [1]:
# %%
# === IMPORTS ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
import joblib
import os

warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
sns.set_style("whitegrid")

print("✅ Imports OK")

# %%
# === CHARGEMENT DU TEST INTERIM ===
print("=" * 60)
print("CHARGEMENT DU TEST SET")
print("=" * 60)

# Charger le test INTERIM (déjà nettoyé, comme train)
df = pd.read_csv("../data/interim/test_dataset_M1_interim.csv")
test_ids = df["id"].copy()
df = df.drop(columns=["id"])

print(f"Dataset shape: {df.shape}")
print(f"Test IDs sauvegardés: {len(test_ids)}")
print(f"Colonnes: {df.columns.tolist()}")

# %%
# === FEATURE ENGINEERING - PARTIE 1 : Prix et Discount ===
print("\n" + "=" * 60)
print("FEATURE ENGINEERING - Prix & Discount")
print("=" * 60)

df["Effective_Discount"] = df["Price"] * df["Discount"] / 100
df["Net_Price"] = df["Price"] * (1 - df["Discount"] / 100)
df["Price_Bucket"] = pd.qcut(df["Price"], 5, labels=False, duplicates="drop")

print("✅ Features prix créées")

# %%
# === FEATURE ENGINEERING - PARTIE 2 : Engagement & Interactions ===
print("\n" + "=" * 60)
print("FEATURE ENGINEERING - Engagement & Interactions")
print("=" * 60)

df["Email_x_Engagement"] = df["Email_Interaction"] * df["Engagement_Score"]
df["Cart_x_Engagement"] = df["Items_In_Cart"] * df["Engagement_Score"]

print("✅ Features engagement créées")

# %%
# === FEATURE ENGINEERING - PARTIE 3 : Segments EDA ===
print("\n" + "=" * 60)
print("FEATURE ENGINEERING - Segments EDA")
print("=" * 60)

# Campaign_Period doit être booléen
df["Campaign_Period"] = df["Campaign_Period"].astype(bool)

# Catégorie High Value
df["HighValue_Category"] = df["Category"].isin([0.0, 1.0, 2.0]).astype(int)


# Distance à la campagne
def day_rel_to_campaign(day, campaign_windows=[(25, 50), (75, 90)]):
    if pd.isna(day):
        return np.nan
    campaign_days = [i for w in campaign_windows for i in range(w[0], w[1] + 1)]
    return min([abs(day - c) for c in campaign_days])


df["Day_to_Campaign"] = df["Day"].apply(day_rel_to_campaign)

print("✅ Features segments créées")

# %%
# === 🚀 NOUVELLES FEATURES - INTERACTIONS CAMPAGNE ===
print("\n" + "=" * 60)
print("🚀 NOUVELLES FEATURES - INTERACTIONS CAMPAGNE")
print("=" * 60)

# 1. Interactions multiplicatives campagne
print("   1. Interactions prix × campagne...")
df["Price_x_Campaign"] = df["Price"] * df["Campaign_Period"].astype(int)
df["Discount_x_Campaign"] = df["Discount"] * df["Campaign_Period"].astype(int)
df["Net_Price_x_Campaign"] = df["Net_Price"] * df["Campaign_Period"].astype(int)

# 2. Interactions engagement × campagne
print("   2. Interactions engagement × campagne...")
df["Engagement_x_Campaign"] = df["Engagement_Score"] * df["Campaign_Period"].astype(int)
df["Items_x_Campaign"] = df["Items_In_Cart"] * df["Campaign_Period"].astype(int)
df["Email_x_Campaign"] = df["Email_Interaction"] * df["Campaign_Period"].astype(int)

# 3. Intensité campagne (proximité temporelle)
print("   3. Intensité campagne...")
df["Campaign_Intensity"] = np.where(
    df["Campaign_Period"], 1 / (df["Day_to_Campaign"] + 1), 0
)

# 4. Ratios comportementaux
print("   4. Ratios comportementaux...")
df["Cart_Efficiency"] = df["Items_In_Cart"] / (df["Reviews_Read"] + 1)
df["Engagement_per_Price"] = df["Engagement_Score"] / (df["Net_Price"] + 1)
df["Email_Impact"] = (
    df["Email_Interaction"] * df["Engagement_Score"] * df["Reviews_Read"]
)

# 5. Segments avancés
print("   5. Segments avancés...")
df["High_Intent"] = (
    (df["Items_In_Cart"] >= 3)
    & (df["Engagement_Score"] >= 2.5)
    & (df["Email_Interaction"] == 1)
).astype(int)

df["Price_Sensitive"] = ((df["Discount"] >= 25) & (df["Net_Price"] <= 200)).astype(int)

df["Premium_Buyer"] = (
    (df["Net_Price"] >= 500) & (df["Device_Type"].isin(["Desktop", "Tablet"]))
).astype(int)

print(f"✅ 13 nouvelles features créées !")

# %%
# === MISSING INDICATORS ===
print("\n" + "=" * 60)
print("CRÉATION DES MISSING INDICATORS")
print("=" * 60)

missing_pct = df.isnull().sum() / len(df) * 100
cols_with_missing = missing_pct[missing_pct > 1].index.tolist()

# ⚠️ IMPORTANT : Exclure Day et Day_to_Campaign pour correspondre au train
cols_with_missing = [
    col for col in cols_with_missing if col not in ["Day", "Day_to_Campaign"]
]

for col in cols_with_missing:
    df[f"{col}_missing"] = df[col].isnull().astype(int)

print(f"✅ {len(cols_with_missing)} missing indicators créés")
if cols_with_missing:
    print(f"   Colonnes: {cols_with_missing}")

# %%
# === IMPUTATION ===
print("\n" + "=" * 60)
print("IMPUTATION DES VALEURS MANQUANTES")
print("=" * 60)

# Valeurs médianes du TRAIN pour imputation cohérente
median_values = {
    "Age": 30.0,
    "Gender": 0.0,
    "Reviews_Read": 3.0,
    "Price": 300.0,
    "Discount": 25.0,
    "Category": 2.0,
    "Items_In_Cart": 3.0,
    "Socioeconomic_Status_Score": 4.5,
    "Engagement_Score": 2.5,
    "AB_Bucket": 3.0,
    "Price_Sine": 0.0,
    "Day": 80.0,
    "Email_Interaction": 0.0,
    "Net_Price": 225.0,
    "Price_Bucket": 2.0,
    "Email_x_Engagement": 0.0,
    "Cart_x_Engagement": 7.5,
    "Day_to_Campaign": 15.0,
}

# Numériques : utiliser les médianes du train
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        fill_value = median_values.get(col, df[col].median())
        df[col].fillna(fill_value, inplace=True)

# Catégorielles : "Unknown"
categorical_cols = df.select_dtypes(include=["object", "bool"]).columns
for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna("Unknown", inplace=True)

print(f"✅ Imputation terminée - NA restants: {df.isnull().sum().sum()}")

# %%
# === RARE CATEGORIES ===
print("\n" + "=" * 60)
print("GESTION DES CATÉGORIES RARES (<1%)")
print("=" * 60)

threshold = 0.01
rare_handling = {}

for col in categorical_cols:
    if col == "Campaign_Period":  # Skip booléen
        continue
    value_counts = df[col].value_counts(normalize=True)
    rare_categories = value_counts[value_counts < threshold].index.tolist()

    if rare_categories:
        df[col] = df[col].apply(lambda x: "Other" if x in rare_categories else x)
        rare_handling[col] = len(rare_categories)

if rare_handling:
    print(f"✅ {len(rare_handling)} colonnes avec catégories rares regroupées")
else:
    print("✅ Aucune catégorie rare détectée")

# %%
# === 🧹 SUPPRESSION FEATURES INUTILES ===
print("\n" + "=" * 60)
print("🧹 NETTOYAGE - SUPPRESSION FEATURES INUTILES")
print("=" * 60)

features_to_remove = ["Effective_Discount"]

features_removed = []
for feat in features_to_remove:
    if feat in df.columns:
        df = df.drop(columns=[feat])
        features_removed.append(feat)

print(f"✅ {len(features_removed)} features supprimées:")
for feat in features_removed:
    print(f"   - {feat}")

print(f"\n📊 Features totales: {len(df.columns)}")

# %%
# === PIPELINE CATBOOST ===
print("\n" + "=" * 60)
print("PRÉPARATION PIPELINE CATBOOST")
print("=" * 60)

cat_features = [
    "Device_Type",
    "Time_of_Day",
    "Payment_Method",
    "Referral_Source",
    "Category",
]

# Convertir en string
for col in cat_features:
    if col in df.columns:
        df[col] = df[col].astype(str)

# Créer X_test - GARDER Campaign_Period et Day comme dans train
cols_to_drop_cb = ["Session_ID"]
X_test_cb = df.drop(columns=cols_to_drop_cb, errors="ignore")

# Indices catégoriels
cat_indices = [
    X_test_cb.columns.get_loc(col) for col in cat_features if col in X_test_cb.columns
]

print(f"✅ Pipeline CatBoost préparé:")
print(f"   X_test shape: {X_test_cb.shape}")
print(f"   Categorical indices: {cat_indices}")
print(f"   Features: {X_test_cb.shape[1]}")

# %%
# === PIPELINE CLASSIC ===
print("\n" + "=" * 60)
print("PRÉPARATION PIPELINE CLASSIC (One-Hot + Scaling)")
print("=" * 60)

# Charger les colonnes du train pour alignement
try:
    train_data = joblib.load("../data/processed/classic_ready.pkl")
    train_columns = train_data["X_train"].columns.tolist()
    print(f"✅ Colonnes train chargées: {len(train_columns)} features")
except:
    print("⚠️ Impossible de charger les colonnes train - génération sans alignement")
    train_columns = None

# Copier données
df_classic = df.copy()

# Convertir Campaign_Period en int
df_classic["Campaign_Period"] = df_classic["Campaign_Period"].astype(int)

# One-hot encoding
cols_onehot = ["Device_Type", "Time_of_Day", "Category"]
df_classic = pd.get_dummies(df_classic, columns=cols_onehot, drop_first=True)

# Label encoding
for col in ["Payment_Method", "Referral_Source"]:
    if col in df_classic.columns:
        le = LabelEncoder()
        df_classic[col] = le.fit_transform(df_classic[col].astype(str))

# Préparer X_test
cols_to_drop = ["Session_ID"]
X_test_cls = df_classic.drop(columns=cols_to_drop, errors="ignore")

# ALIGNER SUR TRAIN SI DISPONIBLE
if train_columns is not None:
    # Ajouter colonnes manquantes
    missing_cols = set(train_columns) - set(X_test_cls.columns)
    for col in missing_cols:
        X_test_cls[col] = 0

    # Supprimer colonnes en trop
    extra_cols = set(X_test_cls.columns) - set(train_columns)
    if extra_cols:
        X_test_cls = X_test_cls.drop(columns=list(extra_cols))

    # Réordonner selon train
    X_test_cls = X_test_cls[train_columns]
    print(f"✅ Test aligné sur train: {X_test_cls.shape}")

# Scaling (fit sur test car pas d'accès au scaler train)
scaler = StandardScaler()
X_test_cls_scaled = scaler.fit_transform(X_test_cls)
X_test_cls = pd.DataFrame(X_test_cls_scaled, columns=X_test_cls.columns)

print(f"✅ Pipeline Classic préparé:")
print(f"   X_test shape: {X_test_cls.shape}")

# %%
# === VÉRIFICATION FINALE ===
print("\n" + "=" * 60)
print("VÉRIFICATION FINALE")
print("=" * 60)

print(f"✅ CatBoost:")
print(f"   Shape: {X_test_cb.shape}")
print(f"   Features: {X_test_cb.shape[1]}")
print(f"   NA: {X_test_cb.isnull().sum().sum()}")

print(f"\n✅ Classic:")
print(f"   Shape: {X_test_cls.shape}")
print(f"   Features: {X_test_cls.shape[1]}")
print(f"   NA: {X_test_cls.isnull().sum().sum()}")

print(f"\n✅ Test IDs: {len(test_ids)}")

print(f"\n📊 Comparaison avec train attendu:")
print(f"   Train attendu: ~56 colonnes (CatBoost)")
print(f"   Test:          {X_test_cb.shape[1]} colonnes (CatBoost)")
if X_test_cb.shape[1] >= 55 and X_test_cb.shape[1] <= 57:
    print("   ✅ PARFAIT ! Nombre de colonnes cohérent")
else:
    print(f"   ⚠️ Vérifier - différence détectée")

# %%
# === SAUVEGARDE ===
print("\n" + "=" * 60)
print("SAUVEGARDE DES DONNÉES PREPROCESSED")
print("=" * 60)

os.makedirs("../data/processed", exist_ok=True)

# CatBoost
catboost_data = {
    "X_test": X_test_cb,
    "test_ids": test_ids,
    "cat_indices": cat_indices,
    "cat_features_names": [col for col in X_test_cb.columns if col in cat_features],
}

joblib.dump(catboost_data, "../data/processed/catboost_ready_test.pkl", protocol=4)
print("✅ CatBoost test: 'catboost_ready_test.pkl'")

# Classic
classic_data = {
    "X_test": X_test_cls,
    "test_ids": test_ids,
    "scaler": scaler,
}

joblib.dump(classic_data, "../data/processed/classic_ready_test.pkl", protocol=4)
print("✅ Classic test: 'classic_ready_test.pkl'")

# Résumé
print("\n" + "=" * 60)
print("RÉSUMÉ FINAL")
print("=" * 60)
print(f"📦 CatBoost test: {X_test_cb.shape}")
print(f"📦 Classic test: {X_test_cls.shape}")
print(f"📦 Test IDs: {len(test_ids)}")
print(f"\n🚀 NOUVELLES FEATURES AJOUTÉES: +13 features")
print("\n🎉 PREPROCESSING TEST SET TERMINÉ !")
print("=" * 60)

✅ Imports OK
CHARGEMENT DU TEST SET
Dataset shape: (6265, 19)
Test IDs sauvegardés: 6265
Colonnes: ['Age', 'Gender', 'Reviews_Read', 'Price', 'Discount', 'Category', 'Items_In_Cart', 'Time_of_Day', 'Email_Interaction', 'Device_Type', 'Payment_Method', 'Referral_Source', 'Socioeconomic_Status_Score', 'Engagement_Score', 'AB_Bucket', 'Price_Sine', 'Session_ID', 'Day', 'Campaign_Period']

FEATURE ENGINEERING - Prix & Discount
✅ Features prix créées

FEATURE ENGINEERING - Engagement & Interactions
✅ Features engagement créées

FEATURE ENGINEERING - Segments EDA
✅ Features segments créées

🚀 NOUVELLES FEATURES - INTERACTIONS CAMPAGNE
   1. Interactions prix × campagne...
   2. Interactions engagement × campagne...
   3. Intensité campagne...
   4. Ratios comportementaux...
   5. Segments avancés...
✅ 13 nouvelles features créées !

CRÉATION DES MISSING INDICATORS
✅ 29 missing indicators créés
   Colonnes: ['Age', 'Gender', 'Reviews_Read', 'Price', 'Discount', 'Category', 'Items_In_Cart', 'T